In [ ]:
#!pip install requests
import requests
from pathlib import Path
import json
import numpy as np 

In [ ]:
api_key = '1f6ed49e0bfb08c3b16a637edf9b2d5c64d5d15d1aa6d940c0bdf54b09f24471'
api_baseUrl = 'https://min-api.cryptocompare.com'
dataPath = Path('data')
dataPath.mkdir(exist_ok=True) # if (!basePath.exists()) 


In [ ]:
def apiGetExchangesNames() :
    global api_key, api_baseUrl
    url = '{0}/data/exchanges/general'.format(api_baseUrl)
    response = requests.get(url, headers={ 'authorization': 'Apikey '+api_key })
    
    # return [exchange for key, exchange in response.json()['Data'].values()]
    exchanges = [ex['Name'] for ex in response.json()['Data'].values()]
    exchanges.sort()
    return exchanges

In [ ]:
def apiGetCrypto(exchange) :
    global api_key, api_baseUrl
    url = '{0}/data/v2/all/exchanges?e={1}'.format(api_baseUrl, exchange)
    response = requests.get(url, headers={ 'authorization': 'Apikey '+api_key })
    return response.json()['Data'][exchange]['pairs']

In [ ]:
def apiGetOHLCV(exchange, pair, period, date, nb=2000) :
    global api_key, api_baseUrl
    (cf, ct) = pair.split('-')
    urls = { 'd':'histoday', 'h':'histohour','m':'histominute' }
    options = [ \
        'fsym=' + cf, \
        'tsym=' + ct, \
        'limit=' + str(nb), \
        'e=' + exchange, \
    ]
    if (date != None) : options.append('toTs=' + str(date))
    url = '{0}/data/{1}?{2}'.format(api_baseUrl, urls[period], '&'.join(options))
    print(url)
    response = requests.get(url, headers={ 'authorization': 'Apikey ' + api_key })
    return response.json()['Data']
    

In [ ]:
def menuExchanges(useApi=False) :
    global dataPath
    exchanges = apiGetExchangesNames() if useApi \
        else [exchange.name for exchange in dataPath.glob('*') if exchange.is_dir()]
    while True :
        for i, exchange in enumerate(exchanges) :
            print("{0}) {1}".format(i, exchange))
        if not useApi :
            print("t) tous")
        choix = input("Choix : ")
        if not useApi and choix.lower() == 't' : return menuExchanges(True)
        try : 
            choix = int(choix)
            if choix>=0 and choix<len(exchanges) : 
                return exchanges[choix]
        except:
            pass

In [ ]:
def menuCrypto(exchange) :
    cryptos = apiGetCrypto(exchange)
    cf = cryptos.keys()
    
    while True :        
        print('From :')
        for c in cf :
            print(c)
        print('* (tous)')
        print('Separateur : /')
        cs = [ c.strip().upper() for c in input('Choix :').split('/') ]
        if '*' in cs : break
        if len([ c for c in cs if c not in cf ]) == 0 : 
            cf = cs
            break
            
    ct = np.unique([c for f in cf for c in cryptos[f]])
    while True :
        print('To :')
        for c in ct :
            print(c)
        print('* (tous)')
        print('Separateur : /')
        cs = [ c.strip().upper() for c in input('Choix :').split('/') ]
        if '*' in cs : break
        if len([ c for c in cs if c not in ct ]) == 0 : 
            ct = cs
            break


    return [ '{0}-{1}'.format(c1, c2) for c1 in cf for c2 in ct if c2 in cryptos[c1] ]


In [ ]:
def updatePair(exchange, pair, period) :
    global dataPath
    if (period not in ['d', 'h', 'm']) : return
    exchangePath = dataPath / exchange
    exchangePath.mkdir(exist_ok=True)
    pairFile = exchangePath / '{0}.{1}'.format(pair, period)
    
    try :
        data = json.loads(pairFile.read_text())
        data = { int(t):d for t,d in data.items() }
    except : 
        data = {}

    date = int(max(data.keys())) if len(data)>0 else None
    count = 2000 if date==None else {'d':10, 'h': 240, 'm': 2000}[period]
    #print(date)
    
    lDate = None
    while True:
        apiData = apiGetOHLCV(exchange, pair, period, lDate, count)
        if (len(apiData)==0) : break
        nData = { \
            nd['time'] : { 'close':nd['close'], 'high':nd['high'], 'low':nd['low'], 'open':nd['open'], 'volumefrom':nd['volumefrom'], 'volumeto':nd['volumeto'] } \
            for nd in apiData if nd['close']>0 and (date==None or nd['time']>date) }
        data = { **data, **nData}
        if len(nData) != len(apiData) : break
        lDate = min(nData.keys())
    #print(len(data))
    #if not pairFile.exists() : pairFile.write_text('{}')
#    if period=="m" : print(data)
    data = dict(sorted(data.items()))
    pairFile.write_text(json.dumps(data))


In [ ]:
exchange = menuExchanges()


In [ ]:
pairs = menuCrypto(exchange)

In [ ]:
for pair in pairs :
    for period in ['d', 'h', 'm'] :
        updatePair(exchange, pair, period)